# Cyclic Polling System with Exhaustive Service and Deterministic Times

This example demonstrates a cyclic polling system with exhaustive service discipline and deterministic arrival and service times. The system uses immediate switchover between classes.

In [ ]:
// Kotlin notebook
import jline.*
import jline.lang.*
import jline.lang.nodes.*
import jline.lang.processes.*
import jline.lang.constant.*
import jline.solvers.mva.*
import jline.solvers.jmt.*
import jline.util.matrix.*

In [ ]:
val model = Network("M[2]/M[2]/1-Exhaustive")

// Block 1: nodes
val source = Source(model, "mySource")
val queue = Queue(model, "myQueue", SchedStrategy.POLLING)
val sink = Sink(model, "mySink")

In [ ]:
// Block 2: classes
val oclass1 = OpenClass(model, "myClass1")
source.setArrival(oclass1, Det(1.0))
queue.setService(oclass1, Det(0.001))

val oclass2 = OpenClass(model, "myClass2")
source.setArrival(oclass2, Det(1.0))
queue.setService(oclass2, Det(0.001))

// Configure polling system
queue.setPollingType(PollingType.EXHAUSTIVE)
queue.setSwitchover(oclass2, Immediate())
queue.setSwitchover(oclass1, Immediate())

In [ ]:
// Block 3: topology
val P = model.initRoutingMatrix()
P.addRoute(oclass1, source, queue, sink)
P.addRoute(oclass2, source, queue, sink)
model.link(P)

In [ ]:
// Solve with MVA (solution is approximate in general for polling systems)
try {
    val mvaSolver = MVA(model)
    val avgTableMVA = mvaSolver.avgTable
    println("MVA Results:")
    avgTableMVA.print()
} catch (e: Exception) {
    println("MVA failed for this polling model: ${e.message}")
    println("This is expected as MVA may not support all polling configurations.")
}

In [ ]:
// Try solving with JMT
try {
    val jmtOptions = JMT.defaultOptions()
    jmtOptions.samples = 100000
    
    val jmtSolver = JMT(model, jmtOptions)
    val avgTableJMT = jmtSolver.avgTable
    println("\nJMT Results:")
    avgTableJMT.print()
} catch (e: Exception) {
    println("\nJMT failed for this polling model: ${e.message}")
    println("This is expected as JMT may not support all polling configurations.")
}

In [ ]:
// Display model configuration summary
println("\n=== Polling System Configuration ===")
println("Model: ${model.name}")
println("Polling Type: Exhaustive")
println("Number of Classes: ${model.numberOfClasses}")
println("Switchover Times: Immediate for both classes")
println("\nClass Configuration:")
println("- Class 1: Deterministic arrivals (rate=1.0), Deterministic service (time=0.001)")
println("- Class 2: Deterministic arrivals (rate=1.0), Deterministic service (time=0.001)")
println("\nSystem Load per Class: λ × E[S] = 1.0 × 0.001 = 0.001")
println("Total System Load: 0.002")
println("\nNote: This is a very light load system with extremely fast service times.")

This example demonstrates:

1. **Cyclic Polling System**: 
   - Single server alternating between two job classes
   - Exhaustive service discipline (serve all jobs in queue before switching)
2. **Deterministic Timing**:
   - Deterministic inter-arrival times (1.0 time units)
   - Deterministic service times (0.001 time units)
   - Immediate switchover times between classes
3. **Solver Challenges**:
   - Polling systems are complex and not all solvers support them
   - MVA and JMT may have limitations with certain polling configurations
   - The combination of deterministic timing and polling can be challenging to analyze
4. **System Characteristics**:
   - Very low utilization (ρ = 0.002 total)
   - Fast service relative to arrival rate
   - Equal treatment of both job classes

Polling systems are important in telecommunications, computer networks, and manufacturing systems where a single server must handle multiple job types or stations.